In [1]:
%%capture
!pip install -r requirements.txt

In [2]:
from huggingface_hub import login
login()

In [3]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)




config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [6]:
%%capture
!pip install faiss-cpu sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Initialize a SentenceTransformer model for embedding
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample documents
documents = ["Experience the luxury of our new line of handcrafted leather bags, designed with both style and functionality in mind.",
"Indulge in the rich aroma of our premium coffee beans, freshly roasted to perfection and sourced from the finest farms around the world.",
"Elevate your kitchen with our sleek and modern stainless steel cookware set, featuring durable construction and innovative designs.",
"Stay connected on the go with our powerful and lightweight laptop, featuring a high-resolution display and lightning-fast processor.",
"Upgrade your entertainment experience with our immersive home theater system, delivering crystal-clear sound and stunning visuals.",
"Discover the perfect balance of comfort and style with our collection of designer sneakers, crafted from premium materials and featuring cutting-edge technology.",
"Pamper your skin with our natural and organic skincare products, formulated with nourishing ingredients to leave you feeling refreshed and rejuvenated.",
"Transform your space with our versatile and stylish furniture pieces, designed to complement any décor and enhance your lifestyle.",
"Indulge in the sweet and savory flavors of our gourmet chocolates, handcrafted with the finest ingredients and packaged in elegant gift boxes.",
"Keep your little ones entertained and engaged with our educational and interactive toys, designed to spark creativity and imagination."]

# Embed documents
doc_embeddings = embedding_model.encode(documents)

# Create a Faiss index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# Create a simple mapping from indices to documents
doc_mapping = {i: doc for i, doc in enumerate(documents)}


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
doc_mapping

{0: 'Experience the luxury of our new line of handcrafted leather bags, designed with both style and functionality in mind.',
 1: 'Indulge in the rich aroma of our premium coffee beans, freshly roasted to perfection and sourced from the finest farms around the world.',
 2: 'Elevate your kitchen with our sleek and modern stainless steel cookware set, featuring durable construction and innovative designs.',
 3: 'Stay connected on the go with our powerful and lightweight laptop, featuring a high-resolution display and lightning-fast processor.',
 4: 'Upgrade your entertainment experience with our immersive home theater system, delivering crystal-clear sound and stunning visuals.',
 5: 'Discover the perfect balance of comfort and style with our collection of designer sneakers, crafted from premium materials and featuring cutting-edge technology.',
 6: 'Pamper your skin with our natural and organic skincare products, formulated with nourishing ingredients to leave you feeling refreshed and 

In [15]:
import asyncio

async def retrieve(query):
    # Generate query embedding
    query_embedding = embedding_model.encode([query])

    # Search in FAISS
    _, indices = index.search(query_embedding, k=2)  # retrieve top 2 documents

    # Fetch relevant documents
    results = [doc_mapping[idx] for idx in indices[0]]
    return results




In [16]:
async def generate_response(query, retrieved_docs):
    # Combine the query with retrieved documents
    messages = [
    {"role": "system", "content": """You are a helpful assistant for an online store. Use the given context to answer
    user queries."""},
    {"role": "user", "content": f"Question: {query}\nContext: {' '.join(retrieved_docs)}"},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]["content"]

In [22]:
async def generate_response_with_html_tags(query, retrieved_docs):
    # Combine the query with retrieved documents
    messages = [
    {"role": "system", "content": """You are a helpful assistant for an online store. Use the given context which are relevant documents retrieved given to you, to answer
    user queries. When using information from the retrieved documents, enclose those portions of your answer within <RAG> and </RAG> tags."""},
    {"role": "user", "content": f"Question: {query}\nContext: {' '.join(retrieved_docs)}"},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]["content"]

In [17]:
# Test the retrieval function
async def main():
    query = "Does this shop offer any skincare products?"
    retrieved_docs = await retrieve(query)
    response = await generate_response(query, retrieved_docs)
    print("Generated answer:", response)

In [18]:
await main()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated answer: Yes, our shop does offer skincare products. We have a range of natural and organic skincare products that are formulated with nourishing ingredients to help leave your skin feeling refreshed and rejuvenated. Would you like to know more about our skincare collection or perhaps find a specific product that suits your skin type?


In [23]:
async def main():
    query = "Does this shop offer any skincare products?"
    retrieved_docs = await retrieve(query)
    response = await generate_response_with_html_tags(query, retrieved_docs)
    print("Generated answer:", response)

In [24]:
await main()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated answer: Based on the provided context, it appears that this shop does offer skincare products. According to the information, they have a collection of natural and organic skincare products that are formulated with nourishing ingredients to provide a refreshing and rejuvenating experience.

<RAG> Pamper your skin with our natural and organic skincare products, formulated with nourishing ingredients to leave you feeling refreshed and rejuvenated. </RAG>

However, it's worth noting that the primary focus of the shop seems to be on designer sneakers, as mentioned in the second part of the context.


In [69]:
async def generate_response_with_tags(query, retrieved_docs):
    # Combine the query with retrieved documents
    messages = [
    {"role": "system", "content": """You are a helpful assistant for an online store. You will do this in 3 steps. 
    step 1. Use the given context which are relevant documents retrieved given to you, to answer
    user queries. 
    step 2.Take a breath for 2 seconds.
    step 3. Now, tag the words or phrases in your response that you got from the context using parentheses and then provide your answer."""},
    {"role": "user", "content": f"Question: {query}\nContext: {' '.join(retrieved_docs)}"},
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]["content"]

In [70]:
async def main(query):
    retrieved_docs = await retrieve(query)
    response = await generate_response_with_tags(query, retrieved_docs)
    print("Generated answer:", response)
    return response
    

In [71]:
query = "Does this shop offer any skincare products?"
response_with_highlights = await main(query)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated answer: (Breath in for 2 seconds, Breath out for 2 seconds)

Yes, this shop offers (skincare products) as mentioned in the context. They offer natural and organic (skincare products) that are formulated with (nourishing ingredients) to provide a refreshed and rejuvenated feeling.


In [72]:
def highlight_words(text, highlight_color):
    """Highlights words or phrases in parentheses with a specified color.

    Args:
        text (str): The input text.
        highlight_color (str): The color code for highlighting (e.g., "\033[31m" for red).

    Returns:
        str: The highlighted text with parentheses removed.
    """

    import re
    pattern = r"\((.*?)\)"

    highlighted_text = re.sub(pattern, lambda x: f"{highlight_color}{x.group(1)}\033[0m", text)
    

    return highlighted_text



In [73]:
highlighted_text = highlight_words(response_with_highlights, "\033[32m")
print(highlighted_text)

Breath in for 2 seconds, Breath out for 2 seconds

Yes, this shop offers skincare products as mentioned in the context. They offer natural and organic skincare products that are formulated with nourishing ingredients to provide a refreshed and rejuvenated feeling.
